In [ ]:
#import all the packages that we will need when working with dataframes and geometry data
import pandas as pd
import glob
import os
from shapely.geometry import Point, Polygon
import numpy as np
from math import radians, cos, sin, asin, sqrt
import datetime
from datetime import datetime
from shapely.geometry.multipolygon import MultiPolygon
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype


In [ ]:
#### set the location of the files - keep in mind these are on L drive so need to be connected to the VPN

path = "L:/EMPoWeR Pilot Study/Data/GPS Data/410002/Canmore/"


In [ ]:
##remove the files if they exist as they will cause problems when running the script multiple times
if os.path.exists(os.path.join(path, "410002_all_points_unclean.csv")):
    os.remove(os.path.join(path, "410002_all_points_unclean.csv"))
if os.path.exists(os.path.join(path, "410002_allpoints_clean.csv")):
    os.remove(os.path.join(path, "410002_allpoints_clean.csv"))
if os.path.exists(os.path.join(path, "410002_allpoints_clean_final.csv")):
    os.remove(os.path.join(path, "410002_allpoints_clean_final.csv"))
if os.path.exists(os.path.join(path, "410002_sleep_points.csv")):
    os.remove(os.path.join(path, "410002_sleep_points.csv"))
if os.path.exists(os.path.join(path, "410002_totalcounts.csv")):
    os.remove(os.path.join(path, "410002_totalcounts.csv"))

In [ ]:
#get a list of all the files in the folder
all_files = glob.glob(os.path.join(path, "*.csv"))

In [ ]:
#read all the files into the dataframe
#if running code over and over, need to only have the path to just the separated gps files
df = pd.concat((pd.read_csv(f, header=None) for f in all_files)).reset_index()

In [ ]:
#These data don't have headers already so lets make some
headers = ['index1','index', 'date(UTC)', 'time(UTC)', 'date(local)', 'time(local)', 'latitude', 'lat(DMM)', 'wrong_longitude', 'lon(DMM)', 'elevation(m)', 'speed(km)']
df.columns = headers

df

In [ ]:
#all the longitudes need to be negative so let's convert by slicing
#this is bc Canmore data are recorded in degrees minutes seconds (N/W) and need to be in decimal degrees (-long)
#first convert the long column to float if it isn't already
df['wrong_longitude'] = df['wrong_longitude'].astype(float)

#create a new column with the longitude converted to negative 
df['longitude'] = (df.loc[:,'wrong_longitude']) * (-1)


#lets get rid of all these unecessary columns that we don't care about
df = df.drop(columns=['index1', 'lat(DMM)', 'wrong_longitude', 'lon(DMM)'])

df


In [ ]:
#combine date(local) & time(local) into a new column called DateTime_Local and then convert to datetime object
df['DateTime_Local'] =  pd.to_datetime(df['date(local)'].astype(str)+' '+ df['time(local)'].astype(str))
df['DateTime_Local'] = pd.to_datetime(df['DateTime_Local'])

#now lets just move it over after all the other times and move longitude next to latitude to make it look nice
column_to_move = df.pop("DateTime_Local")
df.insert(5, "DateTime_Local", column_to_move)

column_to_move = df.pop("longitude")
df.insert(7, "longitude", column_to_move)

df

In [ ]:
#select only the dates that match with the EMA dataset dates. Can be found on "GPS Matching" Excel
#need to use the following code with the index
df.set_index('DateTime_Local', inplace=True)

# slice the data 
From = '2022-04-13'
To   = '2022-04-26'

df = df.loc[From:To,:]

#save to the file
df.to_csv(os.path.join(path, '410002_all_points_unclean.csv'))

df.reset_index(inplace=True)
df

In [ ]:
#run Haversine fcn to get the euclidean distance between data points and store in new column calculating distance in km and mi
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = map(np.radians,[lat1,lon1,lat2,lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


df['distKM'] = \
    haversine(df.latitude.shift(), df.longitude.shift(),
              df.loc[1:, 'latitude'], df.loc[1:, 'longitude'])
df['distMI'] = df['distKM'] * 0.621371
#distKM is in km, distMI is converted to miles
#can double check values for code working with this calculator:https://www.calculator.net/distance-calculator.html
#note that lat lon decimal points round to the 6th decimal point;6th decimal point measures to up to 0.11m error

df


In [ ]:
#calculate the difference in time from each previous timestamp in hh:mm:ss and put in column called 'diff_in_hours'

#make the DateTime_Local the index to run to_series function
df.index = df['DateTime_Local']

#now calculate distance from previous timepoint to next in hour format
df['diff_in_hours'] = df.index.to_series().diff()

#calculate new columns for time between points in mins and secs 
df['durationSecs'] = df['diff_in_hours'].dt.total_seconds()
df['durationMins'] = df['durationSecs'] // 60

df

In [ ]:
#downsample the Canmore 5s data to 20s
df = df.resample('20S').first()

df

In [ ]:
#creates empty rows when resampling, so need to drop these first 
df['index'].replace('', np.nan, inplace=True)
df.dropna(subset=['index'], inplace=True)


In [ ]:
#this tells us how many rows we have left after downsampling and removing those empty rows
df.shape

# Data Cleaning

Now we will pre-process the data by finding erroneous points first by applying our filtering criteria
-We will make sure that speed is <95mph (152.888km/hr)
-We will make sure that elevation is <14410 feet (highest hiking spot in USA)(4392 meters)

-Note that speed is in km in our dataset & elevation in meters

In [ ]:
#create a new column where we will identify if a row is erroneous
df['noise'] = np.nan

#convert both speed and elevation to integers & make the name simpler or following code won't work
df['speed'] = df['speed(km)'].astype(int)
df['elevation'] = df['elevation(m)'].astype(int)

#drop the old column names because its getting messy
df = df.drop(columns=['speed(km)', 'elevation(m)'])

#find where speed is > than our criteria, if it finds one, it will mark '1' in the noise column
df.loc[df.speed >= 152, 'noise'] = 1
#find where elevation is > than our criteria, if it finds one, it will mark '1' in the noise column
df.loc[df.elevation >= 4392, 'noise'] = 1

#this tells us how many noisy points there are on account of speed or elevation outliers
df.loc[df.noise == 1, 'noise'].count()


In [ ]:
#redo haversine & time difference
df.reset_index(drop=True, inplace=True)

df['distKM'] = \
    haversine(df.latitude.shift(), df.longitude.shift(),
              df.loc[1:, 'latitude'], df.loc[1:, 'longitude'])
df['distMI'] = df['distKM'] * 0.621371

df

In [ ]:
#need to also redo the time difference between points now that we resampled
#make the DateTime_Local the index to run to_series function
df.index = df['DateTime_Local']

#now calculate distance from previous timepoint to next in hour format
df['diff_in_hours'] = df.index.to_series().diff()

#calculate new columns for time between points in mins and secs 
df['durationSecs'] = df['diff_in_hours'].dt.total_seconds()
df['durationMins'] = df['durationSecs'] // 60

df

In [ ]:
#now our last filtering criteria
#.0264 is the amount of miles per second a person can travel going 95mph (the max value)
#95/3600 = .0264 mp/s

y = df.durationSecs * .0264
df.loc[df.distMI > y, 'noise'] = 1

#so if the distance from one point to the next is greater than possible based on time from one point to the next, it's labeled as noise

#check to see how many possible erroneous points there are in our dataset
df.loc[df.noise == 1, 'noise'].count()
#df

In [ ]:
#reset index
#df.reset_index(drop=True, inplace=True)
#df

In [ ]:
#if I already ran the code once and have a 1 in the noise column, otherwise need to set noise = 1
noise = df['noise'].count().astype(int)

noiseList = []

for i, row in df.iterrows():
        if noise > 0:
            df.reset_index(drop=True, inplace=True)
            df['distKM'] = \
            haversine(df.latitude.shift(), df.longitude.shift(),
                df.loc[1:, 'latitude'], df.loc[1:, 'longitude'])
            df['distMI'] = df['distKM'] * 0.621371
            df.index = df['DateTime_Local']
        #now calculate distance from previous timepoint to next in hour format
            df['diff_in_hours'] = df.index.to_series().diff()
        #calculate new columns for time between points in mins and secs
            df['durationSecs'] = df['diff_in_hours'].dt.total_seconds()
            df['durationMins'] = df['durationSecs'] // 60
            maxDist = df.durationSecs * .0264
        #if the distance between point A and > B given 95mph, then put 1 in column noise
            df.loc[df.distMI > maxDist, 'noise'] = 1
            noise = df['noise'].count().astype(int)
            noiseList.append(noise)
            df.drop(df.loc[df['noise']==1].index, inplace=True)
        else:
            break
df


In [ ]:
#letas count how much noise total we got rid of and call it 

noiseSum = sum(noiseList)
noiseSum

In [ ]:
#just rerun the distance and time calculation again to make sure its correct
df.reset_index(drop=True, inplace=True)
df['distKM'] = \
     haversine(df.latitude.shift(), df.longitude.shift(),
               df.loc[1:, 'latitude'], df.loc[1:, 'longitude'])
df['distMI'] = df['distKM'] * 0.621371


In [ ]:
#need to also redo the time difference between points now that we resampled
#make the DateTime_Local the index to run to_series function
df.index = df['DateTime_Local']

#now calculate distance from previous timepoint to next in hour format
df['diff_in_hours'] = df.index.to_series().diff()

#calculate new columns for time between points in mins and secs 
df['durationSecs'] = df['diff_in_hours'].dt.total_seconds()
df['durationMins'] = df['durationSecs'] // 60

df

# Fix Resampling
- error in not picking up all the resamples
- remove those & redo haversine & time difference

In [ ]:
df.drop(df[df.durationSecs <20].index, inplace=True)
df

In [ ]:
#need to redo the haversine & time diff AGAIN one last time
#just rerun the distance and time calculation again to make sure its correct
df.reset_index(drop=True, inplace=True)
df['distKM'] = \
     haversine(df.latitude.shift(), df.longitude.shift(),
               df.loc[1:, 'latitude'], df.loc[1:, 'longitude'])
df['distMI'] = df['distKM'] * 0.621371


#need to also redo the time difference between points now that we resampled
#make the DateTime_Local the index to run to_series function
df.index = df['DateTime_Local']

#now calculate distance from previous timepoint to next in hour format
df['diff_in_hours'] = df.index.to_series().diff()

#calculate new columns for time between points in mins and secs 
df['durationSecs'] = df['diff_in_hours'].dt.total_seconds()
df['durationMins'] = df['durationSecs'] // 60

df

In [ ]:
#save the file as th3 cleaned file!!!!!
df.to_csv(os.path.join(path, "410002_allpoints_clean.csv"),index=None)

# Hand remove outliers
- identify very obvious erroneous datapoints
- mark index of item
- remove
- rerun haversine & distance 
- if none, skip all of these next steps

In [ ]:
#drop the index column & recount
df = df.drop('index', axis = 1)
df.insert(1, 'Index', range(1, 1 + len(df)))
df

# Visualization

- Now lets look at these points in using Folium
- Zoom out to look for any outliers and continue to remove
- Otherwise inspect to make sure nothing looks off

In [ ]:
import folium 
import requests

#starts the map at Iowa City
allmap = folium.Map(location=[41.6611, -91.5302], zoom_start=11)

#apply df points on the map, using lat, lon rows & Index column as a popup
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius=5, color='red', popup=row['Index']).add_to(allmap), axis=1)

#display map
allmap

# Hand remove outliers part 2
- remove using above map index, mark down in Excel
- rerun haversine & distance 
- if none, skip all of these next steps

In [ ]:
#drop the rows with the indexes you mark as outliers based on visual inspection from map
#first make Index the index
df.set_index('Index', inplace = True)

#then drop those specific indexes you noted
df.drop([13677, 7132, 7131, 7130, 7129, 13738, 27196, 13728, 13727, 27195, 27194, 27193, 27192, 7133, 27191, 27190, 27189, 27188, 27187, 9042, 9041, 9040, 7326, 7328, 7327, 7325, 16453, 29785, 29784, 29783, 16452, 29782, 16451, 16450, 3589, 16449, 2307, 2306, 13674, 2250, 2249, 2252, 2251, 13675, 13676, 13673], inplace = True)

df.reset_index(inplace = True)
#then redo the Index colum to recount

df['Index'] = range(1, 1 + len(df))
df

In [ ]:
#visualize again to make sure all gone

#starts the map at Iowa City
allmap2 = folium.Map(location=[41.6611, -91.5302], zoom_start=11)

#apply df points on the map, using lat, lon rows & Index column as a popup
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius=5, color='red', popup=row['Index']).add_to(allmap2), axis=1)

#display map
allmap2

In [ ]:
#if all looks good, rerun haversine & time diff and save file
df.reset_index(drop=True, inplace=True)
df['distKM'] = \
     haversine(df.latitude.shift(), df.longitude.shift(),
               df.loc[1:, 'latitude'], df.loc[1:, 'longitude'])
df['distMI'] = df['distKM'] * 0.621371


#need to also redo the time difference between points now that we resampled
#make the DateTime_Local the index to run to_series function
df.index = df['DateTime_Local']

#now calculate distance from previous timepoint to next in hour format
df['diff_in_hours'] = df.index.to_series().diff()

#calculate new columns for time between points in mins and secs 
df['durationSecs'] = df['diff_in_hours'].dt.total_seconds()
df['durationMins'] = df['durationSecs'] // 60

df

In [ ]:
#save the file as the final cleaned file
df.to_csv(os.path.join(path, "410002_allpoints_clean_final.csv"),index=None)

# Sleep Points Dataframe
- Now we will get a new df with just the sleep points
- Perform DBSCAN to get the sleep cluster

In [ ]:
#now lets get the participants home location by only selecting the coordinates between 2am-5am and creating a new df with them
#set the datetime index and get the points between 2 and 5
df.set_index('DateTime_Local', inplace=True)
sleep_points = df.between_time('02:00', '05:00')
sleep_points.shape

#can save to file for double checking and/or doing DBSCAN in ArcGIS
#sleep_points.to_csv(os.path.join(path, "tff_sleep_points.csv"),index=None)


In [ ]:
#visualize sleep points on map to make sure nothing weird going on
sleep_points.reset_index(inplace = True)

sleep_points.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius=5, color='blue', popup=row['DateTime_Local']).add_to(allmap2), axis=1)

allmap2

# Data Clustering
- Reasoning for why DBSCAN performs better than k-means for lat/lon data: https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

- We want to perform DBSCAN on our sleep points data
- The idea is to identify the "home point" cluster of our data - where participants are located between 2am-5am
- Once we identify this cluster using DBSCAN, we will get the median center point of this cluster 
- This median center home point (in lat,lon) will be compared to our orig. df with a polygon surrounding this coordinate of 50m. Any point within 50m of this polygon will be characterized as "pt is spending time at home" and anything outside will be captured as "pt is spending time outside home"
- Finally we will calculate the proportion of time spent at home versus away from home in the participant's whole dataset

In [ ]:
# we just want the lat/lon points of the sleep df converted to a np matrix
coords = sleep_points[['latitude', 'longitude']].to_numpy()
coords

In [ ]:
#import modules for scatterplots & DBSCAN later
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

#run DBSCAN
#haversine parameter requires distances are in radians
kms_per_radian = 6371.0088

#epsilon = the max distance (50 meters or .05 km in this sample) that points can be from each other to be considered a cluster.
epsilon = .05 / kms_per_radian

#min_samples = min. cluster size (everything else gets classified as noise)
#in this dataset the points are collected every 20 secs so must spent at least 1 hr somewhere to be a cluster at home = 3600/20 = 180 points in an hour
min_sample = 180

db = DBSCAN(eps=epsilon, min_samples=min_sample, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_

#number of clusters
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])

print('Number of clusters: {}'.format(num_clusters))

In [ ]:
#we want to get the point nearest to this cluster's center (cluster 0) and store the lat/long coordinates for later 
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

#Find the point in each cluster that is closest to its centroid
centermost_points = []
for cluster in clusters.iteritems():
    if len(cluster[1]) >= min_sample:
        centermost_points.append(get_centermost_point(cluster[1]))
        

In [ ]:
# unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
lats, lons = zip(*centermost_points)

# from these lats/lons create a new df of one representative point for each cluster (not -1 or outliers)
rep_points = pd.DataFrame({'longitude':lons, 'latitude':lats})

# pull row from original data set where lat/lon match the lat/lon of each row of representative points
#if we wanted to find this point in our overall dataset it makes it easier to see the index & all the other details

rs = rep_points.apply(lambda row: df[(df['latitude']==row['latitude']) & (df['longitude']==row['longitude'])].iloc[0], axis=1)

#save to file with 6 decimal points
rs.to_csv(os.path.join(path, '410002_sleeppoints.csv'), float_format='%.6f', encoding='utf-8')

centermost_points

In [ ]:
#visualize the sleep points cluster and the center point on the cluster in a scatterplot to make sure looks correct
fig, ax = plt.subplots(figsize=[10, 6])
sleep_points_scatter = ax.scatter(sleep_points['longitude'], sleep_points['latitude'], c='purple', edgecolors='black')
rs_scatter = ax.scatter(rs['longitude'], rs['latitude'], c='red', edgecolor='black')
ax.set_title('Sleep Points before and after reduction')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend([sleep_points_scatter, rs_scatter], ['Sleep Points Unclustered', 'Cluster Center'], loc='upper right')
plt.show()

In [ ]:
#reset index if you need to
df.reset_index(inplace=True)
df

# Calculate Basic Metrics
- Find how many data points there are per day
- Find the total distances travelled per day
- Calculate # of points found at home/outside of home

In [ ]:
#count how many points there are per day
#add a new column in our original (cleaned) df
df['count_perday'] = 1

df_counts = df.groupby(by=df['DateTime_Local'].dt.date).count()[['count_perday']].reset_index()

#also add in a column with participant ID because we need that!
df_counts['SubjectID'] = "410002"
#move it over to the left for prettiness
column_to_move = df_counts.pop("SubjectID")
df_counts.insert(0, "SubjectID", column_to_move)

df_counts


In [ ]:
#this will tell you how many total distance traveled (in miles) per day
df_mi_sum = df.groupby(by=df['DateTime_Local'].dt.date).sum()[['distMI']].reset_index()
df_mi_sum

#in the future should probably include other filtering logic so that if days have lots of points but normal "jiggle" in signal

In [ ]:
#concat this into the new df (called df_counts) where we will store all this participant's data
df_counts = pd.merge(df_counts, df_mi_sum, on='DateTime_Local', how='outer')
df_counts

In [ ]:
#now lets find the distance FROM that sleep center point to everywhere else by day
#lets print out the lat/lon of that point
rs

#take from it the lat/lon and put in the code below (copy & paste)

In [ ]:
#rerun haversine from distance from everypoint TO this mean center point
#then manually input latitude above for lat1 below and longitude above for lon1 below
def haversine(row):
    lon1 = -91.535132
    lat1 = 41.667101
    lon2 = row['longitude']
    lat2 = row['latitude']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * np.arcsin(sqrt(a)) 
    km = 6367 * c
    return km 

#add it to our orig. dataframe so we get this for each point
df['distance_from_HomeMI'] = df.apply(lambda row: haversine(row), axis=1) * 0.62137119 #converted to miles by *.62137119, otherwise km
df['distance_from_HomeMeter'] = df['distance_from_HomeMI'] * 1609.344 #converted from miles to meters

df

In [ ]:
#get the max or greatest distance from home per day

df_miFromHome_max = df.groupby(df['DateTime_Local'].dt.date)['distance_from_HomeMI'].agg('max').reset_index()
df_miFromHome_max

In [ ]:
#add to the df_counts df
df_counts = pd.merge(df_counts, df_miFromHome_max, on='DateTime_Local', how='outer')
df_counts

In [ ]:
#now the last thing to find how many points within 50m of home per day or "points at home"
#if distance from home <= 50 meters or 0.0310686 miles, then it is inside home, otherwise outside

df['at_home'] = np.nan
df.loc[df.distance_from_HomeMI <= 0.0310686, 'at_home'] = 1
df

#save this df and replace the previous allpoints_clean df
#NOTE= you need to check if you did visual inspection or not to decide which file name to rewrite (clean or clean_final)
df.to_csv(os.path.join(path, "410002_allpoints_clean_final.csv"),index=None)

In [ ]:
#now group by day and sum all the points within home!
df_points_atHome = df.groupby(by=df['DateTime_Local'].dt.date).sum()[['at_home']].reset_index()
df_points_atHome

In [ ]:
#concat to the df_counts df
df_counts = pd.merge(df_counts, df_points_atHome, on='DateTime_Local', how='outer')
df_counts

In [ ]:
#lastly, calculate % of time spent at home per day
df_counts['%time_at_home'] = ((df_counts['at_home']) / (df_counts['count_perday'])) *100

#rename all the columns names so they are easier to understand
df_counts.rename({"distMI": "Total_DistMI_SUM", 
           "distance_from_HomeMI": "MAX_distancefromhome_MI"}, 
          axis = "columns", inplace = True)
#save this excel as the final metrics for this participant
df_counts.to_csv(os.path.join(path, '410002_totalcounts.csv'))

df_counts

In [ ]:
#get the average of time spent at home across all days
df_counts['AVG_%time_at_home'] = df_counts.loc[:, '%time_at_home'].mean()

df_counts

In [ ]:
#get the average distances travelled 
df_counts['AVG_disMI'] = df_counts.loc[:, 'Total_DistMI_SUM'].mean()

df_counts

In [ ]:
#save this excel as the final metrics for this participant
df_counts.to_csv(os.path.join(path, '410002_totalcounts.csv'))


# 